## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.73,135.87,50.99,80,57,10.20,broken clouds
1,1,Pangnirtung,CA,66.15,-65.71,50.00,81,100,8.05,overcast clouds
2,2,Tessalit,ML,20.20,1.01,75.18,72,98,14.88,moderate rain
3,3,Rikitea,PF,-23.12,-134.97,73.36,75,0,2.04,clear sky
4,4,Chunoyar,RU,57.45,97.32,52.61,97,81,3.65,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 74
What is the maximum temperature you would like for your trip? 88


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tessalit,ML,20.20,1.01,75.18,72,98,14.88,moderate rain
10,10,Poum,NC,-20.23,164.02,75.43,86,100,7.36,overcast clouds
12,12,Hithadhoo,MV,-0.60,73.08,80.64,69,100,1.43,overcast clouds
13,13,Amahai,ID,-3.33,128.92,76.37,89,63,4.09,broken clouds
16,16,Brumadinho,BR,-20.14,-44.20,75.52,42,29,10.69,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tessalit,ML,20.20,1.01,75.18,72,98,14.88,moderate rain
10,10,Poum,NC,-20.23,164.02,75.43,86,100,7.36,overcast clouds
12,12,Hithadhoo,MV,-0.60,73.08,80.64,69,100,1.43,overcast clouds
13,13,Amahai,ID,-3.33,128.92,76.37,89,63,4.09,broken clouds
16,16,Brumadinho,BR,-20.14,-44.20,75.52,42,29,10.69,scattered clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tessalit,ML,75.18,moderate rain,20.20,1.01,
10,Poum,NC,75.43,overcast clouds,-20.23,164.02,
12,Hithadhoo,MV,80.64,overcast clouds,-0.60,73.08,
13,Amahai,ID,76.37,broken clouds,-3.33,128.92,
16,Brumadinho,BR,75.52,scattered clouds,-20.14,-44.20,
22,Butaritari,KI,81.28,scattered clouds,3.07,172.79,
24,Dingle,PH,76.75,broken clouds,11.00,122.67,
28,Hilo,US,83.28,mist,19.73,-155.09,
35,Ponta Do Sol,PT,74.71,clear sky,32.67,-17.10,
37,Kokopo,PG,75.70,overcast clouds,-4.35,152.26,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found...skipping') 

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
final_hotel_df = hotel_df.dropna()
final_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Hithadhoo,MV,80.64,overcast clouds,-0.60,73.08,Scoop Guest House
13,Amahai,ID,76.37,broken clouds,-3.33,128.92,HOTEL ISABELA
16,Brumadinho,BR,75.52,scattered clouds,-20.14,-44.20,Ville de Montagne Hotel
22,Butaritari,KI,81.28,scattered clouds,3.07,172.79,Isles Sunset Lodge
24,Dingle,PH,76.75,broken clouds,11.00,122.67,SEGAYA PROPERTY
...,...,...,...,...,...,...,...
532,Shawville,CA,83.77,broken clouds,45.60,-76.48,Shawville Hotel
533,Tottori,JP,74.75,broken clouds,35.50,134.23,Hotel Monarque Tottori
534,Yangjiang,CN,80.42,broken clouds,21.85,111.97,HUALUXE Yangjiang City Center
535,Kearney,US,87.71,clear sky,40.70,-99.08,"Holiday Inn Kearney, an IHG Hotel"


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
final_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = final_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.05)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))